In [80]:
import unittest

class TestAnalysis(unittest.TestCase):
    def test_cumulative_sum(self):
        test_file_name = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        df = load_to_memory(test_file_name)
        use_melt(df)
        print(use_melt(df))
        
        self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 20742904
#         self.assertEqual(df[df['Country/Region'] == 'Zimbabwe'], 0)
#             & df['Date'] == ['2020-08-13'] == 500000
        
        
unittest.main(argv=[''], verbosity=2, exit=False)

test_cumulative_sum (__main__.TestAnalysis) ... 

Using Melt Function
Using Melt Function
            Country/Region       Date   Cases  Cumulative Sum
148149         Afghanistan 2021-07-06  127464        20742904
148150             Albania 2021-07-06  132544        24802238
148151             Algeria 2021-07-06  142447        32820101
148152             Andorra 2021-07-06   13991         2905567
148153              Angola 2021-07-06   39375         6178651
...                    ...        ...     ...             ...
148423             Vietnam 2021-07-06   22341         1103416
148424  West Bank and Gaza 2021-07-06  314780        53675169
148425               Yemen 2021-07-06    6931         1222752
148426              Zambia 2021-07-06  167132        18280206
148427            Zimbabwe 2021-07-06   57963         8142767

[279 rows x 4 columns]


ERROR

ERROR: test_cumulative_sum (__main__.TestAnalysis)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Library/Python/3.8/site-packages/pandas/core/indexes/base.py", line 3080, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas/_libs/index.pyx", line 70, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 101, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 4554, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 4562, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'Date'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "<ipython-input-80-f426da9e68f5>", line 11, in test_cumulative_sum
    self.assertTrue(df[df['Country/Region'] == 'Afghanistan'] & df[df['Date'] == ['2020-07-06']]) == 207

In [25]:
import pandas as pd 
from pull_covid_data import pull_covid_data
from datetime import date

pull_covid_data()

def load_to_memory(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    return data

time_series = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
dataframe = load_to_memory(time_series)

def parse_jhu_timeseries_to_dataframe(df):
    column_constant = 4
    melt_df = pd.melt(df, id_vars=['Country/Region'], value_vars=list(df.columns[column_constant:]))
    melt_df.columns = ['Country/Region', 'Date', 'Cases']
    
    #Grouping by Country/Region and adding column "Cumulative Sum" as a result of groupby
    axis_constant = 0
    melt_df['Cumulative Sum'] = melt_df.groupby(['Country/Region'])['Cases'].cumsum(axis=axis_constant)
    
    #Changing date to proper format
    melt_df['Date'] = pd.to_datetime(melt_df['Date'])
    
    # 279 is the number of unique Country/Regions for the DataFrame
    return melt_df.tail(279)

#Test data files 
cases_file = '/Users/kaleighjaeger/Scripps/Test_Cases.csv'
deaths_file = '/Users/kaleighjaeger/Scripps/Test_Deaths.csv'

def create_test_file(filename):
    data = pd.read_csv(f'{filename}', header=0)
    edited_data = data.head(1)
    return data

def merge_cases_and_deaths(cases_file, deaths_file):
    cases = parse_jhu_timeseries_to_dataframe(create_test_file(f'{cases_file}'))
    cases.columns = ['Country/Region', 'Date', 'Cases', 'Cumulative Cases']
    deaths = parse_jhu_timeseries_to_dataframe(create_test_file(f'{deaths_file}'))
    deaths.columns = ['Country/Region', 'Date', 'Deaths', 'Cumulative Deaths']
    merged_file = pd.merge(cases,deaths, on=['Date', 'Country/Region'])
    return merged_file

def calculate_first_value(df):
    nonzero_constant = 0
    idx = df.Cases.ne(nonzero_constant).idxmax()
    deaths_idx = df.Deaths.ne(nonzero_constant).idxmax()
    df['dead_firstDate'] = df.loc[deaths_idx]['Date']
    df['confirmed_firstDate'] = df.loc[idx]['Date']
    return df

def calculate_first_date():
    df = merge_cases_and_deaths(cases_file, deaths_file)
    df = df.groupby(['Country/Region']).apply(
        calculate_first_value
    )
    return df

def calculate_percent_increase():
    data = calculate_first_date()
    rounded_constant = 2
    data['Percent Increase (Cases)'] = data['Cases'].pct_change().round(rounded_constant)
    return data

def calculate_rolling_average():
    df = calculate_percent_increase()
    data = df.sort_values(by = ['Country/Region', 'Date'])
    #Need to filter for each country using GroupBy 
    rolling_average_constant = 3
    data['Rolling Average'] = df.groupby('Country/Region').rolling(rolling_average_constant)['Cases'].mean().reset_index(drop=True)
#     data['Rolling Average'] = df.groupby('Country/Region').rolling(rolling_average_constant, center=True)['Cases'].mean().reset_index(drop=True)
    return data
    
    
calculate_rolling_average()


Here is the status update of COVID-19: 

On branch master
Your branch is up to date with 'origin/master'.

nothing to commit, working tree clean


,Country/Region,Date,Cases,Cumulative Cases_x,Deaths,Cumulative Cases_y,dead_firstDate,confirmed_firstDate,Percent Increase (Cases),Rolling Average
0,Afghanistan,2020-01-22,0,0,0,0,2020-01-26,2020-01-24,NaN,NaN
3,Afghanistan,2020-01-23,0,0,0,0,2020-01-26,2020-01-24,NaN,1.000000
6,Afghanistan,2020-01-24,1,1,0,0,2020-01-26,2020-01-24,inf,3.666667
9,Afghanistan,2020-01-25,2,3,0,0,2020-01-26,2020-01-24,inf,5.333333
12,Afghanistan,2020-01-26,3,6,1,1,2020-01-26,2020-01-24,2.00,1.000000
15,Afghanistan,2020-01-27,4,10,1,2,2020-01-26,2020-01-24,3.00,3.666667
18,Afghanistan,2020-01-28,4,14,1,3,2020-01-26,2020-01-24,1.00,6.000000
21,Afghanistan,2020-01-29,5,19,2,5,2020-01-26,2020-01-24,0.67,NaN
24,Afghanistan,2020-01-30,5,24,2,7,2020-01-26,2020-01-24,0.00,0.666667
27,Afghanistan,2020-01-31,6,30,3,10,2020-01-26,2020-01-24,0.00,3.333333


In [5]:
def load_to_memory(filename):
    data = pd.read_csv(f'{filename}', header=0) 
    return data

time_series = '/Users/kaleighjaeger/Scripps/epidemiology/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
load_to_memory(time_series)


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/3/21,8/4/21,8/5/21,8/6/21,8/7/21,8/8/21,8/9/21,8/10/21,8/11/21,8/12/21
0,NaN,Afghanistan,33.939110,67.709953,0,0,0,0,0,0,...,148572,148933,149361,149810,149810,149810,150778,151013,151291,151563
1,NaN,Albania,41.153300,20.168300,0,0,0,0,0,0,...,133211,133310,133442,133591,133730,133912,133981,134201,134487,134761
2,NaN,Algeria,28.033900,1.659600,0,0,0,0,0,0,...,175229,176724,178013,179216,180356,181376,182368,183347,184191,185042
3,NaN,Andorra,42.506300,1.521800,0,0,0,0,0,0,...,14766,14797,14809,14836,14836,14836,14836,14873,14891,14908
4,NaN,Angola,-11.202700,17.873900,0,0,0,0,0,0,...,43070,43158,43269,43487,43592,43662,43747,43890,43998,44174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274,NaN,Vietnam,14.058324,108.277199,0,2,2,2,2,2,...,174461,181756,189066,193381,205656,215560,224894,232937,241543,246568
275,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,317264,317404,317534,317703,317703,317999,317999,318181,318435,319223
276,NaN,Yemen,15.552727,48.516388,0,0,0,0,0,0,...,7086,7096,7104,7131,7131,7165,7187,7198,7213,7233
277,NaN,Zambia,-13.133897,27.849332,0,0,0,0,0,0,...,197123,197791,198455,199135,199662,200049,200201,200830,201342,201770


In [12]:
parse_jhu_timeseries_to_dataframe(dataframe)

,Country/Region,Date,Cases,Cumulative Sum
0,Afghanistan,2020-01-22,0,0
1,Albania,2020-01-22,0,0
2,Algeria,2020-01-22,0,0
3,Andorra,2020-01-22,0,0
4,Angola,2020-01-22,0,0
...,...,...,...,...
158746,Vietnam,2021-08-12,246568,5289681
158747,West Bank and Gaza,2021-08-12,319223,65381488
158748,Yemen,2021-08-12,7233,1483142
158749,Zambia,2021-08-12,201770,25293716
